Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print('shape: ', shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

shape:  [16, 7, 7, 16]
shape:  [10000, 7, 7, 16]
shape:  [10000, 7, 7, 16]


In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.155478
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 50: 1.860599
Minibatch accuracy: 37.5%
Validation accuracy: 39.9%
Minibatch loss at step 100: 1.058730
Minibatch accuracy: 68.8%
Validation accuracy: 66.2%
Minibatch loss at step 150: 1.888742
Minibatch accuracy: 37.5%
Validation accuracy: 57.6%
Minibatch loss at step 200: 0.614933
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 250: 2.073089
Minibatch accuracy: 43.8%
Validation accuracy: 76.5%
Minibatch loss at step 300: 0.912263
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 350: 0.629259
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
Minibatch loss at step 400: 0.562807
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.588235
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 500: 0.794743
Minibatch accuracy: 62.5%
Validation accuracy: 80.7%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME') 
       
    shape = hidden.get_shape().as_list()
    print('shape: ', shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

shape:  [16, 7, 7, 16]
shape:  [10000, 7, 7, 16]
shape:  [10000, 7, 7, 16]


In [22]:
def train_conv_nn(num_steps = 9001):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print('Initialized')
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [23]:
%time train_conv_nn()

Initialized
Minibatch loss at step 0: 3.557634
Minibatch accuracy: 0.0%
Validation accuracy: 13.7%
Minibatch loss at step 50: 1.720976
Minibatch accuracy: 37.5%
Validation accuracy: 45.1%
Minibatch loss at step 100: 0.934424
Minibatch accuracy: 75.0%
Validation accuracy: 71.8%
Minibatch loss at step 150: 1.680914
Minibatch accuracy: 37.5%
Validation accuracy: 70.6%
Minibatch loss at step 200: 0.663779
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 250: 2.087906
Minibatch accuracy: 50.0%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.662129
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 350: 0.408635
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.586443
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.467042
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.934678
Minibatch accuracy: 62.5%
Validation accuracy: 81.5%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [27]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  # Model.
  def model(data):
    # input image 28 * 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # pool1 input 24 * 24 -> 28 - 5 + 1
    pool1 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
    # conv2 input 12 * 12 -> 24 / 2
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    # pool2 input 8 * 8 -> 12 - 5 + 1
    pool2 = tf.nn.avg_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
    # hidden input 4 * 4 -> 8 / 2
    shape = pool2.get_shape().as_list()
    print('shape: ', shape)
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

shape:  [16, 4, 4, 16]
shape:  [10000, 4, 4, 16]
shape:  [10000, 4, 4, 16]


In [26]:
%time train_conv_nn()

Initialized
Minibatch loss at step 0: 2.545836
Minibatch accuracy: 12.5%
Validation accuracy: 10.8%
Minibatch loss at step 50: 1.621408
Minibatch accuracy: 50.0%
Validation accuracy: 53.2%
Minibatch loss at step 100: 1.254033
Minibatch accuracy: 62.5%
Validation accuracy: 65.5%
Minibatch loss at step 150: 2.063340
Minibatch accuracy: 31.2%
Validation accuracy: 70.5%
Minibatch loss at step 200: 0.645922
Minibatch accuracy: 75.0%
Validation accuracy: 71.3%
Minibatch loss at step 250: 1.893095
Minibatch accuracy: 50.0%
Validation accuracy: 75.8%
Minibatch loss at step 300: 0.940664
Minibatch accuracy: 68.8%
Validation accuracy: 77.1%
Minibatch loss at step 350: 0.646241
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 400: 0.494081
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 450: 0.577264
Minibatch accuracy: 87.5%
Validation accuracy: 77.6%
Minibatch loss at step 500: 1.018396
Minibatch accuracy: 68.8%
Validation accuracy: 80.0%
M

In [30]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
    
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  # Model.
  def model(data, keep_prob):
    # input image 28 * 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # pool1 input 24 * 24 -> 28 - 5 + 1
    pool1 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
    # conv2 input 12 * 12 -> 24 / 2
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    # pool2 input 8 * 8 -> 12 - 5 + 1
    pool2 = tf.nn.avg_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
    # hidden input 4 * 4 -> 8 / 2
    shape = pool2.get_shape().as_list()
    print('shape: ', shape)
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    dropout1 = tf.nn.dropout(hidden, keep_prob) 
    return tf.matmul(dropout1, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
  # Optimizer.
  decay_step = 4000
  decay_rate = 0.95
  learning_rate = tf.train.exponential_decay(0.05, global_step, decay_step, decay_rate, staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1))

shape:  [16, 4, 4, 16]
shape:  [10000, 4, 4, 16]
shape:  [10000, 4, 4, 16]


In [31]:
%time train_conv_nn()

Initialized
Minibatch loss at step 0: 3.133455
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.307692
Minibatch accuracy: 12.5%
Validation accuracy: 29.0%
Minibatch loss at step 100: 1.468091
Minibatch accuracy: 56.2%
Validation accuracy: 46.8%
Minibatch loss at step 150: 2.398591
Minibatch accuracy: 31.2%
Validation accuracy: 64.7%
Minibatch loss at step 200: 1.270297
Minibatch accuracy: 62.5%
Validation accuracy: 60.0%
Minibatch loss at step 250: 2.180530
Minibatch accuracy: 43.8%
Validation accuracy: 68.2%
Minibatch loss at step 300: 1.258348
Minibatch accuracy: 68.8%
Validation accuracy: 71.3%
Minibatch loss at step 350: 0.917234
Minibatch accuracy: 68.8%
Validation accuracy: 70.8%
Minibatch loss at step 400: 1.223495
Minibatch accuracy: 62.5%
Validation accuracy: 74.0%
Minibatch loss at step 450: 0.825484
Minibatch accuracy: 75.0%
Validation accuracy: 70.1%
Minibatch loss at step 500: 1.237457
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Mi